# Example api

In [1]:
import os
os.chdir('../src')

In [2]:
os.getcwd()

'D:\\ds\\autoDL-CV\\src'

In [3]:
from dotenv import load_dotenv

In [4]:
load_dotenv(dotenv_path='../example/.env')

True

In [5]:
from api.logs import enable_logs
from api.data.segmentations import SegmentationDataset
from logging import StreamHandler

In [7]:
enable_logs(StreamHandler())

In [8]:
from api.data.segmentations.types import ImageMask

In [10]:
segm_dataset = SegmentationDataset(
    r'F:\dataset\ph2',
    ImageMask, # аннотация для наследника
    (256, 256),
    (0.7, 0.2, 0.1),
    16,
)

 100% |█████████████████| 200/200 [1.2s elapsed, 0s remaining, 169.4 samples/s]         


INFO - Split dataset train size: 140, valid size: 40, test size: 20
INFO - Creating a dataset
DEBUG - Dataset created with params, dataset dir: F:\dataset\ph2, classes: {0: 'background', 1: 'target'}, batch size: 16


In [11]:
from api.optimize import HPOptimizer

In [12]:
from api.models.segmentations import  get_encoders, get_pretrained_weights, get_models, get_torch_optimizers, get_losses, get_optimize_scores
from api.optimize import get_hp_optimizers
# создать отдельный модуль для получения возможных параметров

In [13]:
print('models: ', get_models())
print('encoders: ', get_encoders())
print('pretrained_weights', get_pretrained_weights())
print('optimizers', get_torch_optimizers())
print('losses', get_losses())
print('hp optimizers', get_hp_optimizers())
print('optimize scores', get_optimize_scores())

models:  ['Unet', 'UnetPlusPlus', 'MAnet', 'Linknet', 'FPN', 'PSPNet', 'DeepLabV3', 'DeepLabV3Plus', 'PAN']
encoders:  ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'resnext50_32x4d', 'resnext101_32x4d', 'resnext101_32x8d', 'resnext101_32x16d', 'resnext101_32x32d', 'resnext101_32x48d', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn107', 'dpn131', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'inceptionresnetv2', 'inceptionv4', 'efficientnet-b0', 'efficientnet-b1', 'efficientnet-b2', 'efficientnet-b3', 'efficientnet-b4', 'efficientnet-b5', 'efficientnet-b6', 'efficientnet-b7', 'mobilenet_v2', 'xception', 'timm-efficientnet-b0', 'timm-efficientnet-b1', 'timm-efficientnet-b2', 'timm-efficientnet-b3', 'timm-efficientnet-b4', 'timm-efficientnet-b5', 'timm-efficientnet-b6', 'timm-e

In [14]:
from api.optimize import HyperParamsSegmentation

In [15]:
params = HyperParamsSegmentation(
    strategy='TPESampler',
    architectures=['UnetPlusPlus'],
    encoders=['mobilenet_v2'],
    pretrained_weights=['imagenet'],
    loss_fns=['JaccardLoss'],
    optimizers=['AdamW', 'RMSprop'],
    epoch_range=(5, 5),
    lr_range=(0.001, 0.003),
    optimize_score='loss',
)

In [17]:
hp_optimizer = HPOptimizer(
    hyper_params=params,
    num_trials=1,
    device='GPU',
    dataset=segm_dataset
)

DEBUG - Create hp optimizer with params: strategy: TPESampler num_trials: 1 num_classes: 2 device: cuda optimize_score: {'strategy': 'TPESampler', 'architectures': ['UnetPlusPlus'], 'encoders': ['mobilenet_v2'], 'pretrained_weights': ['imagenet'], 'loss_fns': ['JaccardLoss'], 'optimizers': ['AdamW', 'RMSprop'], 'epoch_range': (5, 5), 'lr_range': (0.001, 0.003), 'optimize_score': 'loss'}  hps: 


In [30]:
from config import get_settings

In [38]:
tensorboard_logs_path = get_settings().TENSORBOARD_LOGS_PATH.as_posix()

In [40]:
tensorboard_logs_path

'../../tmp/tensorboard'

In [49]:
%reload_ext tensorboard

In [52]:
tensorboard --logdir tensorboard_logs_path

Launching TensorBoard...

In [18]:
hp_optimizer.optimize()

INFO - Train models started
[I 2023-02-18 22:24:36,795] A new study created in memory with name: no-name-94b94901-6fe9-4ad0-8e14-46a04e79d768
DEBUG - Create tensorboard logger, path: ../../tmp/28b8cb61538440f1a831eb7abfa845bf/tensorboard/2276587
DEBUG - Dataset updated
INFO - Model 2276587_UnetPlusPlus_mobilenet_v2_imagenet_RMSprop_JaccardLoss_lr=0,0016781637723177554 trained with test loss 0.08877936005592346
DEBUG - Add predictions for model: 2276587_UnetPlusPlus_mobilenet_v2_imagenet_RMSprop_JaccardLoss_lr=0,0016781637723177554


 200 [9.4s elapsed, 20.4 samples/s]   


DEBUG - Save weights model: 2276587_UnetPlusPlus_mobilenet_v2_imagenet_RMSprop_JaccardLoss_lr=0,0016781637723177554, path: ../../tmp/28b8cb61538440f1a831eb7abfa845bf/weights/2276587.pt
[I 2023-02-18 22:25:26,474] Trial 0 finished with value: 0.14474747578303018 and parameters: {'optimizer': 'RMSprop', 'architecture': 'UnetPlusPlus', 'encoders': 'mobilenet_v2', 'pretrained_weight': 'imagenet', 'lr': 0.0016781637723177554, 'loss': 'JaccardLoss', 'num_epoch': 5}. Best is trial 0 with value: 0.14474747578303018.
INFO - Study statistics:
INFO - Number of finished trials: 1
INFO - Number of pruned trials: 0
INFO - Number of complete trials: 1
INFO - Best trial:
INFO - Value: 0.14474747578303018
INFO - Params: 
INFO - optimizer: RMSprop
INFO - architecture: UnetPlusPlus
INFO - encoders: mobilenet_v2
INFO - pretrained_weight: imagenet
INFO - lr: 0.0016781637723177554
INFO - loss: JaccardLoss
INFO - num_epoch: 5
INFO - Train models is over


In [25]:
hp_optimizer.stats_models

,_id,name,architecture,encoder,pretrained_weight,lr,optimizer,loss_fn,num_epoch,loss,fbeta_score,f1_score,iou_score,accuracy,positive_predictive_value,sensitivity
0,199cb3d,199cb3d_UnetPlusPlus_mobilenet_v2_imagenet_RMS...,UnetPlusPlus,mobilenet_v2,imagenet,0.001366,RMSprop,JaccardLoss,5,0.081,0.951,0.951,0.911,0.961,0.950,0.959
1,abd41ce,abd41ce_UnetPlusPlus_mobilenet_v2_imagenet_RMS...,UnetPlusPlus,mobilenet_v2,imagenet,0.001850,RMSprop,JaccardLoss,5,0.140,0.902,0.902,0.839,0.926,0.921,0.914
2,c82410b,c82410b_UnetPlusPlus_mobilenet_v2_imagenet_RMS...,UnetPlusPlus,mobilenet_v2,imagenet,0.001545,RMSprop,JaccardLoss,5,0.105,0.927,0.927,0.872,0.948,0.944,0.924


In [26]:
# создать модуль по управлению моделей

In [8]:
import fiftyone as fo

In [9]:
fo.launch_app(segm_dataset.fo_dataset)

Connected to FiftyOne on port 5151 at localhost.
If you are not connecting to a remote session, you may need to start a new session and specify a port


Dataset:          ImageMask
Media type:       image
Num samples:      200
Selected samples: 0
Selected labels:  0
Session URL:      http://localhost:5151/